In [27]:
import numpy as np
import pandas as pd
from datetime import datetime, date, time, timedelta

df = pd.read_csv('bigdata.csv')
# df = pd.read_csv('bigdata_time.csv')

In [28]:
df.head()

,Time,PRICE,SIZE,EXCHANGE
0,10:00:00.009,135.14,100,V
1,10:00:00.010,135.14,100,V
2,10:00:00.013,135.13,100,D
3,10:00:00.015,135.13,300,D
4,10:00:00.016,135.13,150,D


In [31]:
# df['Time'] = datetime.strptime(df['Time'].iloc[1], '%H:%M:%S.%f')
j = 0
for i in df['Time']:
    df['Time'][j] = datetime.strptime(i, '%H:%M:%S.%f')
    j += 1

# df['Time'] =df['Time'].strftime('%H:%M:%S.%f')
# n = df['Time'].strftime('%H:%M:%S.%f')
# n
# df['Time']

C:\Users\ANDREY.FENOMEN\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [32]:
df.to_csv('bigdata_time.csv')

In [33]:
df.shape

(9993, 4)

In [34]:
timer = df['Time'][9992] - df['Time'][0]
# t1.year, t1.month, t1.day, t1.hour, t1.minute, t1.second
timer

datetime.timedelta(0, 958, 597000)

##### Количество сделок за промежуток времени 12:01:08.104 и 12:01:09.103

Функция по расчёту максимального количества сделок

In [35]:
def maxtrans(start, stop):
    return df[(df['Time'] >= start) & (df['Time'] <= stop)]['EXCHANGE'].count()

In [36]:
s = maxtrans(datetime(1900, 1, 1, 10, 1, 8, 104), datetime(1900, 1, 1, 10, 1, 9, 103))
s

23

##### Групировка количества сделок по биржам

Функция группировки сделок по биржам

In [38]:
def drouped(start, stop):
    return df[(df['Time'] >= start) & (df['Time'] <= stop)].groupby(df['EXCHANGE'])['EXCHANGE'].count()

# drouped = df[(df['Time'] >= datetime(1900, 1, 1, 10, 1, 8, 104)) & (df['Time'] <= datetime(1900, 1, 1, 10, 1, 9, 103))].groupby(df['EXCHANGE'])['EXCHANGE'].count()

##### Предварительный "грубый" поиск максимального количества сделог (шаг 1 секунда)

In [39]:
delta = timedelta(seconds = 1)

timestart = df['Time'][0]
timeend = df['Time'][9992]
transcount = 0

bigtimestart = 0

for i in range(timer.seconds):
    timestep = timestart + delta
    if timestep <= timeend:
        n = maxtrans(timestart, timestep)
        if n > transcount:
            transcount = n
            bigtimestart = timestart
            print(transcount, bigtimestart)
    else:
        break
    timestart = timestep
                

10 1900-01-01 10:00:00.009000
35 1900-01-01 10:00:06.009000
56 1900-01-01 10:00:46.009000
95 1900-01-01 10:01:03.009000
479 1900-01-01 10:02:00.009000


In [40]:
def shaping(start, stop):
    n = df[(df['Time'] >= start) & (df['Time'] <= stop)].shape[0]
    return n

##### Уточняющий расчет в районе промежутся с нужным выбросом большого количества сделок. (шаг 100 микросекунд)

In [42]:
def prog_micro(bigtimestart, delta, microdel):
    delta = timedelta(seconds = delta)
    transcount = 0
    bigtime = 0
    delta1 = timedelta(microseconds = microdel)
#     print(delta1)
    start1 = bigtimestart - delta
#     print(start1)
    stop1 = bigtimestart + delta * 2
#     print(stop1)
    timer1 = stop1 - start1
#     print(timer1.seconds)
    
#     print(int(timer1/delta1))
    
    for i in range(int(timer1/delta1)):
        time_interval = start1 + delta1 + delta
#         print('time_interval',time_interval)
        if time_interval <= stop1:
            n = drouped(start1, time_interval).sum()
            if n > transcount:
                transcount = n
                bigtime = start1 + delta1
                bigend = time_interval
#                 print(transcount, bigtime, bigend, bigend-bigtime)
        else:
            break
        start1 = start1 + delta1
    print(transcount, bigtime, bigend, bigend-bigtime)
    return bigtime, bigend

In [43]:
prog_micro(bigtimestart, 1, 100)
# %time

516 1900-01-01 10:02:00.734000 1900-01-01 10:02:01.734000 0:00:01


(datetime.datetime(1900, 1, 1, 10, 2, 0, 734000),
 datetime.datetime(1900, 1, 1, 10, 2, 1, 734000))

##### Максимальное найденое число сделок за 1 секунду равняется 516 в промежутке 10:02:00.734000 10:02:01.734000

максимальное количество сделок (в количестве 516) в течение одной секунды было между 10:02:00.734000 и 10:02:01.734000

##### Выводим полученное количество сделок, сгруппированное по биржам:

In [47]:
drouped(datetime(1900, 1, 1, 10, 2, 0, 734000), datetime(1900, 1, 1, 10, 2, 1, 734000))

EXCHANGE
B      1
D     47
J      2
K    133
P     41
Q    239
X      9
Y      1
Z     43
Name: EXCHANGE, dtype: int64